In [1]:
import torch
import random
import numpy as np
from functools import partial

In [2]:
from temp.data.gen import parallel_generate_problem, parallel_generate_solutions
from temp.data.problem import setcover
from temp.data.info import ModelInfo
from temp.data.dataset import ModelGraphDataset
from temp.data.aug import parallel_augment_info


from temp.solver.utils import solve_inst


In [3]:
PRE_TRAIN_DIR = "temp/pre_train"
TRAIN_DIR = "temp/train"
VALID_DIR = "temp/valid"

In [4]:
# parallel_generate_problem(setcover, PRE_TRAIN_DIR, 10, 10)
# parallel_generate_solutions(PRE_TRAIN_DIR, 10)

# parallel_generate_problem(setcover, TRAIN_DIR, 10, 10)
# parallel_generate_solutions(TRAIN_DIR, 10)

# parallel_generate_problem(setcover, VALID_DIR, 10, 10)
# parallel_generate_solutions(VALID_DIR, 10)

In [5]:
m = setcover()
vals = solve_inst(m)
info = ModelInfo.from_model(m)
info.var_info.sols = np.array([[m.objVal] + vals])

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 22.4.0 22E252)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 100 rows, 200 columns and 1991 nonzeros
Model fingerprint: 0x367be945
Variable types: 0 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 187.0000000
Presolve time: 0.00s
Presolved: 100 rows, 200 columns, 1991 nonzeros
Variable types: 0 continuous, 200 integer (200 binary)

Root relaxation: objective 7.000526e+01, 222 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Nod

In [6]:
from temp.data.dataset import info_to_data
d = info_to_data(info)
aug = partial(parallel_augment_info, n=100)

In [11]:
d = ModelGraphDataset(PRE_TRAIN_DIR, augment=parallel_augment_info)

data = d[0][1]
var_feature_size = data.var_node_features.size(-1)
con_feature_size = data.con_node_features.size(-1) 

In [8]:
import pandas as pd
CFG = pd.read_excel("temp/trained_models/setcover_model_configs.xlsx", index_col=0).loc[0].to_dict()
CFG["num_epochs"] = 5
CFG["num_layers"] = 4
CFG["hidden"] = 8

In [9]:
from temp.model.utils import get_model
from temp.model.trainer import train

In [13]:
model_name, model, criterion, optimizer, scheduler = get_model(".", var_feature_size, con_feature_size, n_batches=1, **CFG)

In [14]:
scheduler.total_steps = 10000

In [ ]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed()
    np.random.seed(worker_seed)
    random.seed(worker_seed)

from torch_geometric.loader import DataLoader
pretrain_loader = DataLoader(d_pretrain, batch_size=8, shuffle=True, worker_init_fn=seed_worker, generator=torch.Generator().manual_seed(0))
train_loader = DataLoader(d, batch_size=8, shuffle=True, worker_init_fn=seed_worker, generator=torch.Generator().manual_seed(0))
val_loader = DataLoader(valid_d, batch_size=8, shuffle=True, worker_init_fn=seed_worker, generator=torch.Generator().manual_seed(0))

In [ ]:
train(model_name, model, criterion, optimizer, scheduler, pretrain_loader, train_loader, val_loader, config, False, "./")